# Lab 7B: Foundry Tools - MCP Integration 

**Mission: Give your agent superpowers with the Model Context Protocol!**

In this lab, we'll connect our Foundry agent to the **Microsoft Learn MCP Server** - giving it access to the entire universe of Microsoft documentation. Imagine asking your agent about Azure Cosmos DB and getting answers grounded in official docs!

## What is MCP?

The **Model Context Protocol (MCP)** is like a universal adapter for AI tools. Instead of building custom integrations for every service, MCP provides a standard way for agents to:

| Without MCP | With MCP |
|-------------|----------|
| Custom code per tool | Standard protocol |
| Hardcoded integrations | Plug-and-play tools |
| Rebuild for each service | Connect once, use anywhere |

## What You'll Learn

1. 🔧 **Configure MCPTool** - Connect to the Microsoft Learn MCP server
2. ✅ **Handle Approvals** - Process MCP tool approval requests

## The Microsoft Learn MCP Server

A **free, public MCP server** that gives agents access to:
- 📚 All Microsoft documentation (Azure, .NET, Microsoft 365, etc.)
- 🔍 Semantic search across docs
- 📄 Full article fetching
- 💻 Code sample search

**Endpoint:** `https://learn.microsoft.com/api/mcp`

## Related Labs

| Lab | Topic |
|-----|-------|
| [lab-7b-ai-gateway.ipynb](lab-7b-ai-gateway.ipynb) | Deploy APIM for MCP governance |
| [lab-7b-private-catalog.ipynb](lab-7b-private-catalog.ipynb) | Create tool catalog with API Center |
| [lab-7b-self-hosted-mcp.ipynb](lab-7b-self-hosted-mcp.ipynb) | Deploy custom MCP on Azure Functions |

## Prerequisites

- Completed **Lab 1A** (Landing Zone) and **Lab 1B** (Project Spoke)
- `.env` file with project credentials

> 💡 **No authentication needed!** The Microsoft Learn MCP Server is free and public.

In [ ]:
!pip install azure-ai-projects==2.0.0b2 azure-ai-agents azure-identity -q

In [ ]:
import os

# Load the .env file from the workspace root
env_file = '/workspaces/getting-started-with-foundry/.env'
with open(env_file) as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            os.environ[key] = value

# Verify required configuration from Lab 1B
required_vars = ["SPOKE_ENDPOINT", "SPOKE_PROJECT", "APIM_CONNECTION", "MODEL_NAME"]
for var in required_vars:
    if os.environ.get(var):
        print(f"✅ {var} is set")
    else:
        print(f"❌ {var} is not set - Complete Lab 1B first")

# Optional: GitHub token for testing
if os.environ.get("GITHUB_TOKEN"):
    print(f"✅ GITHUB_TOKEN is set")
else:
    print(f"⚠️  GITHUB_TOKEN not set (optional)")

# Construct the project endpoint from spoke outputs
SPOKE_ENDPOINT = os.environ.get("SPOKE_ENDPOINT", "")
SPOKE_PROJECT = os.environ.get("SPOKE_PROJECT", "")
APIM_CONNECTION = os.environ.get("APIM_CONNECTION", "")
MODEL_NAME = os.environ.get("MODEL_NAME", "gpt-4o")

if SPOKE_ENDPOINT:
    account_host = SPOKE_ENDPOINT.replace("https://", "").replace(".cognitiveservices.azure.com/", "")
    PROJECT_ENDPOINT = f"https://{account_host}.services.ai.azure.com/api/projects/{SPOKE_PROJECT}"
    GATEWAY_MODEL = f"{APIM_CONNECTION}/{MODEL_NAME}"
    print(f"\n📋 Project Endpoint: {PROJECT_ENDPOINT}")
    print(f"📋 Gateway Model: {GATEWAY_MODEL}")

---

## Part 1: Understanding MCPTool Configuration 🔧

The `MCPTool` class is your spacecraft's docking adapter - it connects your Foundry agent to any MCP server!

### Key Parameters

| Parameter | Description | Example |
|-----------|-------------|---------|
| `server_label` | Unique identifier (alphanumeric + underscore) | `"microsoft_learn"` |
| `server_url` | The MCP server endpoint | `"https://learn.microsoft.com/api/mcp"` |
| `require_approval` | When to require human approval | `"always"`, `"never"`, or dict |
| `project_connection_id` | Connection for stored credentials | `"my-github-connection"` |
| `allowed_tools` | Limit which tools to expose | `["microsoft_docs_search"]` |

In [3]:
# MCPTool Configuration Example
# Think of this as programming your agent's "universal translator" for tools!

example_config = """
mcp_tool = MCPTool(
    server_label="microsoft_learn",                    # Your nickname for this server
    server_url="https://learn.microsoft.com/api/mcp", # The MCP endpoint
    require_approval="always",                         # Safety first! 🛡️
)
"""

print("🚀 MCPTool Configuration Pattern:")
print(example_config)
print("Approval Modes:")
print('  • "always"  - Houston, we need approval! (safest)')
print('  • "never"   - Autopilot mode (use with caution)')
print('  • {"never": ["tool1"]}  - Auto-approve specific tools')
print('  • {"always": ["tool2"]} - Only these need mission control')

🚀 MCPTool Configuration Pattern:

mcp_tool = MCPTool(
    server_label="microsoft_learn",                    # Your nickname for this server
    server_url="https://learn.microsoft.com/api/mcp", # The MCP endpoint
    require_approval="always",                         # Safety first! 🛡️
)

Approval Modes:
  • "always"  - Houston, we need approval! (safest)
  • "never"   - Autopilot mode (use with caution)
  • {"never": ["tool1"]}  - Auto-approve specific tools
  • {"always": ["tool2"]} - Only these need mission control


### Approval Modes

| Mode | Value | Behavior |
|------|-------|----------|
| Always | `"always"` | Every tool call requires approval |
| Never | `"never"` | No approval needed (use with caution) |
| Selective | `{"never": [...], "always": [...]}` | Fine-grained control |

---

## Part 2: Connect to Microsoft Learn MCP Server 📚

Time to give our agent access to the entire Microsoft documentation universe!

The **Microsoft Learn MCP Server** provides three powerful tools:

| Tool | Description |
|------|-------------|
| `microsoft_docs_search` | Search across all Microsoft docs |
| `microsoft_docs_fetch` | Fetch complete article content |
| `microsoft_code_sample_search` | Find code examples |

> 🆓 **Free & Public!** No API keys, no authentication, no limits (within reason).

In [4]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool
from azure.identity import DefaultAzureCredential
from openai.types.responses.response_input_param import McpApprovalResponse, ResponseInputParam

# Initialize project client using the spoke project endpoint
project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=PROJECT_ENDPOINT
)

# Get OpenAI client for Responses API
openai_client = project_client.get_openai_client()

print(f"✅ Connected to AI Project: {SPOKE_PROJECT}")

✅ Connected to AI Project: project-h6cmx3


In [6]:
# 🚀 Create MCPTool for Microsoft Learn - your agent's knowledge booster!
mcp_tool = MCPTool(
    server_label="microsoft_learn",
    server_url="https://learn.microsoft.com/api/mcp",
    require_approval="always",  # We'll handle approvals manually first
)

# Create tools list with proper typing
tools: list[Tool] = [mcp_tool]

print("📚 MCPTool configured for Microsoft Learn!")
print(f"   Server Label: microsoft_learn")
print(f"   Server URL: https://learn.microsoft.com/api/mcp")
print(f"   Require Approval: always")
print("\n🎯 Available tools:")
print("   • microsoft_docs_search - Search documentation")
print("   • microsoft_docs_fetch - Get full articles")
print("   • microsoft_code_sample_search - Find code examples")

📚 MCPTool configured for Microsoft Learn!
   Server Label: microsoft_learn
   Server URL: https://learn.microsoft.com/api/mcp
   Require Approval: always

🎯 Available tools:
   • microsoft_docs_search - Search documentation
   • microsoft_docs_fetch - Get full articles
   • microsoft_code_sample_search - Find code examples


In [7]:
# 🤖 Create our Documentation Expert Agent!
agent = project_client.agents.create_version(
    agent_name="docs-expert",  # Note: hyphens OK, underscores NOT allowed
    definition=PromptAgentDefinition(
        model=GATEWAY_MODEL,
        instructions="""You are a Microsoft Documentation Expert with access to all of Microsoft Learn.

Your mission:
🔍 Search official Microsoft documentation to answer questions
📚 Provide accurate, up-to-date information from docs.microsoft.com
💻 Find relevant code samples when asked

Always:
- Use the microsoft_docs_search tool to find relevant documentation
- Cite your sources with article titles and URLs
- If you can't find something, say so honestly

You're like having the entire Microsoft documentation library in your pocket! 🚀""",
        tools=tools,
    ),
    description="Agent with Microsoft Learn documentation access via MCP.",
)

print(f"✅ Documentation Expert Agent created!")
print(f"   Name: {agent.name}")
print(f"   Version: {agent.version}")
print(f"   Model: {GATEWAY_MODEL}")
print(f"\n🎯 Ready to answer questions using official Microsoft docs!")

✅ Documentation Expert Agent created!
   Name: docs-expert
   Version: 1
   Model: landing-zone-apim/gpt-4.1-mini

🎯 Ready to answer questions using official Microsoft docs!


---

## Part 3: Handling MCP Approval Requests 🛡️

When `require_approval="always"`, the agent asks for permission before using any tool. 
Think of it like mission control approving each maneuver!

**The Flow:**
1. 🙋 User asks a question
2. 🤖 Agent decides to use an MCP tool
3. 🔐 Agent returns an `mcp_approval_request`
4. ✅ You approve (or deny) the request
5. 🚀 Agent proceeds with the tool call

In [8]:
# Create a conversation thread to maintain context across multiple interactions
conversation = openai_client.conversations.create()
print(f"✅ Conversation created: {conversation.id}")

✅ Conversation created: conv_72975d16c0507a63004cAmZKjIb4q1VYRlHUbfzVyWEsNVBrNs


In [9]:
# 🚀 Let's ask about something space-related in Azure!
response = openai_client.responses.create(
    conversation=conversation.id,
    input="How do I use Azure Cosmos DB for storing satellite telemetry data?",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print(f"📡 Response received!")
print(f"   Response ID: {response.id}")
print(f"   Status: {response.status}")
print(f"\n🔍 Checking for approval requests...")

📡 Response received!
   Response ID: resp_72975d16c0507a63006978ee414fa481909e6d8c40ce082bd6
   Status: completed

🔍 Checking for approval requests...


In [10]:
# 🛡️ Process MCP approval requests - Mission Control style!
input_list: ResponseInputParam = []

for item in response.output:
    if item.type == "mcp_approval_request":
        print(f"🔐 APPROVAL REQUESTED")
        print(f"   Server: {item.server_label}")
        print(f"   Request ID: {item.id}")
        
        if item.server_label == "microsoft_learn" and item.id:
            # Auto-approve for this demo (it's just searching docs!)
            input_list.append(
                McpApprovalResponse(
                    type="mcp_approval_response",
                    approval_request_id=item.id,
                    approve=True,
                )
            )
            print(f"   ✅ Mission Control: APPROVED! 🚀")

if input_list:
    print(f"\n📋 Total approvals granted: {len(input_list)}")
else:
    print("ℹ️ No approval requests - agent responded directly")

🔐 APPROVAL REQUESTED
   Server: microsoft_learn
   Request ID: mcpr_72975d16c0507a63006978ee43366881908c8bdac4cad06fcd
   ✅ Mission Control: APPROVED! 🚀

📋 Total approvals granted: 1


In [11]:
# 🚀 Send approvals and get the final response!
if input_list:
    response = openai_client.responses.create(
        input=input_list,
        previous_response_id=response.id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )

print("📚 Documentation Expert Response:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

📚 Documentation Expert Response:
To use Azure Cosmos DB for storing satellite telemetry data, you can leverage its strengths in handling high-velocity IoT data, real-time analytics, global distribution, and flexible data models. Here's a summary and guidance based on official Microsoft documentation:

1. **Use Case Fit**: Azure Cosmos DB is well-suited for IoT and telemetry data scenarios, including satellite telemetry, because it can ingest bursts of data from sensors/devices, process streaming data for real-time insights, and store data for ad hoc querying and batch analytics.

2. **Typical Architecture**:
   - Use **Azure Event Hubs** to ingest high throughput telemetry data from satellites.
   - Use **Azure Stream Analytics** or similar services to perform real-time processing and analytics on the incoming telemetry data.
   - Store the telemetry data in **Azure Cosmos DB** for low latency querying and flexible data access.
   - Use the **change feed** feature of Cosmos DB to track

### 🎮 Try More Questions!

Run the cell below with different questions to see your agent search Microsoft docs:

In [12]:
# 🎮 Helper function to ask questions with auto-approval
def ask_docs_expert(question: str) -> str:
    """Ask the documentation expert a question with auto-approval."""
    # Create new conversation
    conv = openai_client.conversations.create()
    
    # Send question
    resp = openai_client.responses.create(
        conversation=conv.id,
        input=question,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )
    
    # Auto-approve any MCP requests
    approvals: ResponseInputParam = []
    for item in resp.output:
        if item.type == "mcp_approval_request" and item.id:
            approvals.append(McpApprovalResponse(
                type="mcp_approval_response",
                approval_request_id=item.id,
                approve=True,
            ))
    
    # Get final response if we had approvals
    if approvals:
        resp = openai_client.responses.create(
            input=approvals,
            previous_response_id=resp.id,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        )
    
    return resp.output_text

# Try it out! 🚀
answer = ask_docs_expert("What is Azure Functions and how do I create my first function?")
print("📚 Answer:")
print(answer)

📚 Answer:
Azure Functions is a serverless compute service that lets you run event-driven code without having to explicitly provision or manage infrastructure. Functions are blocks of code triggered by events such as HTTP requests, timer schedules, file uploads, messages from queues, and more. This allows you to implement your system's logic in small, readily available pieces of code.

You can create your first Azure Function using multiple tools like:

1. Azure Developer CLI (azd)
2. Azure CLI command line
3. Visual Studio Code
4. Visual Studio

Each of these tools has quickstart tutorials that guide you through creating and deploying your first function in less than five minutes.

For example, if you use Visual Studio to create a C# Azure Function:
- You create a new Azure Functions project.
- Choose your runtime and trigger type (e.g., HTTP trigger).
- Write your function code (e.g., handling HTTP requests).
- Test the function locally.
- Deploy your function app to Azure.

Visual St

---

## 🧹 Cleanup

In [32]:
# try:
#     project_client.agents.delete_version(agent_name=agent.name, version=agent.version)
#     print(f"🗑️ Agent deleted: {agent.name} (version {agent.version})")
# except Exception as e:
#     print(f"Note: Could not delete agent: {e}")
# 
# openai_client.close()
# project_client.close()
# print("✅ Clients closed")

---

## 🎉 Mission Complete!

You've successfully given your agent access to the Microsoft documentation!

### What You Learned

| Concept | Key Takeaway |
|---------|--------------|
| **MCPTool** | Universal adapter for connecting agents to tools |
| **Microsoft Learn MCP** | Free access to all MS docs - no auth needed! |
